In [1]:
# import os
# os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(0)

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.objectives import categorical_crossentropy

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 75.0% of memory, CuDNN 4007)
/usr/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [3]:
%time (X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

CPU times: user 1.29 s, sys: 587 ms, total: 1.88 s
Wall time: 1.88 s
60000 train samples
10000 test samples


In [4]:
%time P = np.load('P.npy') # load pre-computed joint probabilities

CPU times: user 928 µs, sys: 1.53 s, total: 1.53 s
Wall time: 1.98 s


In [5]:
import theano
from theano.tensor.extra_ops import fill_diagonal
from theano import tensor as T
from keras import backend as K

batch_size = 5000

# P is the joint probabilities for this batch (Keras loss functions call this y_true)
# activations is the low-dimensional output (Keras loss functions call this y_pred)
def tsne(P, activations):
    # v = length(network{end}.bias_upW) - 1
    v = K.shape(activations)[1] - 1.
    
    # sum_act = sum(activations .^ 2, 2)
    sum_act = K.sum(K.square(activations), axis=1)
    # Q = (1 + (bsxfun(@plus, sum_act, bsxfun(@plus, sum_act', -2 * activations * activations')) ./ v)) .^ -((v + 1) / 2)
    Q = K.pow(1 + ((sum_act + (K.transpose(sum_act) + -2 * activations * K.transpose(activations))) / v), -((v + 1) / 2))
    # Q(1:n+1:end) = 0
    fill_diagonal(Q, 0) # Theano-only
    # Q = Q ./ sum(Q(:))
    Q /= K.sum(Q)
    # Q = max(Q, eps);
    Q = K.maximum(Q, K.epsilon())
    # C = sum(sum(P{1} .* log((P{1} + eps) ./ (Q + eps))))
    C = K.sum(K.sum(P * K.log((P + K.epsilon()) / (Q + K.epsilon()))))
    return C

In [6]:
model = Sequential()
model.add(Dense(500, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(2000))
model.add(Activation('relu'))
model.add(Dense(2))

sgd = SGD()
%time model.compile(loss=tsne, optimizer=sgd)

CPU times: user 3.31 s, sys: 116 ms, total: 3.42 s
Wall time: 3.47 s


In [7]:
Y_train = P.reshape(X_train.shape[0], -1)
print(X_train.shape)
print(Y_train.shape)

(60000, 784)
(60000, 5000)


In [8]:
model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=20,
          verbose=2)

Epoch 1/20


ValueError: GpuElemwise. Input dimension mis-match. Input 3 (indices start at 0) has shape[1] == 2, but the output's size on that axis is 5000.
Apply node that caused the error: GpuElemwise{Composite{(i0 + (i1 + (i2 * i3 * i4)))},no_inplace}(GpuDimShuffle{x,0}.0, GpuDimShuffle{x,0}.0, CudaNdarrayConstant{[[-2.]]}, GpuElemwise{Add}[(0, 0)].0, GpuDimShuffle{1,0}.0)
Toposort index: 57
Inputs types: [CudaNdarrayType(float32, row), CudaNdarrayType(float32, row), CudaNdarrayType(float32, (True, True)), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix)]
Inputs shapes: [(1, 5000), (1, 5000), (1, 1), (5000, 2), (2, 5000)]
Inputs strides: [(0, 1), (0, 1), (0, 0), (2, 1), (1, 2)]
Inputs values: ['not shown', 'not shown', CudaNdarray([[-2.]]), 'not shown', 'not shown']
Outputs clients: [[HostFromGpu(GpuElemwise{Composite{(i0 + (i1 + (i2 * i3 * i4)))},no_inplace}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created.